In [1]:
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd

from tqdm import tqdm

In [3]:
Xs = np.load("./embed/books_train_review_en_X.npy")
Xt = np.load("./embed/books_train_review_ja_X.npy")
y = np.load("./embed/books_train_review_y.npy")

In [65]:
def serialize_example(elsa_src, elsa_tgt, label):
    elsa_src = tf.keras.preprocessing.sequence.pad_sequences([elsa_src], dtype=elsa_src.dtype, maxlen=20, padding="post")
    elsa_tgt = tf.keras.preprocessing.sequence.pad_sequences([elsa_tgt], dtype=elsa_tgt.dtype, maxlen=20, padding="post")
    #assert elsa_src.shape == elsa_tgt.shape == (1, 20, 2248)
    feature = {
        "src":   tf.train.Feature(float_list=tf.train.FloatList(value=elsa_src.flatten().tolist())),
#        "src_len":  tf.train.Feature(int64_list=tf.train.Int64List(value=[len(elsa_src)])),
        "tgt":   tf.train.Feature(float_list=tf.train.FloatList(value=elsa_tgt.flatten().tolist())),
#        "tgt_len": tf.train.Feature(int64_list=tf.train.Int64List(value=[len(elsa_tgt)])),
        "label":  tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example.SerializeToString()

In [66]:
with tf.python_io.TFRecordWriter("test.tfrecord") as writer:
    for i in tqdm(range(len(y))):
        example = serialize_example(Xs[i], Xt[i], y[i])
        writer.write(example)

100%|██████████| 2000/2000 [00:32<00:00, 63.69it/s]


In [69]:
def parse_example(example):
    features = tf.parse_single_example(
        example,
        features={
            'src': tf.io.VarLenFeature(tf.float32),
#            'src': tf.FixedLenFeature([], tf.float32, default_value=0.0),
#            'src_len': tf.FixedLenFeature([], tf.int64, default_value=0),
            'tgt': tf.io.VarLenFeature(tf.float32),
#            'tgt': tf.FixedLenFeature([], tf.float32, default_value=0.0),
#            'tgt_len': tf.FixedLenFeature([], tf.int64, default_value=0),            
            'label': tf.FixedLenFeature([], tf.int64, default_value=0),
        })    
    return features

dataset = tf.data.TFRecordDataset(['test.tfrecord']).map(parse_example)

In [71]:
for data in dataset.take(1):
    print(repr(data))

{'src': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f66285dfb38>, 'tgt': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f66286dcda0>, 'label': <tf.Tensor: id=834, shape=(), dtype=int64, numpy=0>}


In [72]:
np.all(data["src"].values.numpy().reshape((1, 20, -1)) ==  tf.keras.preprocessing.sequence.pad_sequences([Xs[0]], dtype="float32", maxlen=20, padding="post"))

True

In [73]:
def func(feature):
    src = tf.reshape(feature["src"].values, (20, -1)) 
    tgt = tf.reshape(feature["tgt"].values, (20, -1)) 
    label = feature["label"]
    return {"src": src, "tgt": tgt, "label": label}

In [74]:
for feature in dataset.map(func).take(1):
    print(repr(feature))

{'src': <tf.Tensor: id=872, shape=(20, 2248), dtype=float32, numpy=
array([[-0.0222198 , -0.03705665,  0.130663  , ..., -0.01368509,
        -0.05116319,  0.01393876],
       [-0.06223782, -0.13405626,  0.1653006 , ...,  0.00108687,
        -0.04647795,  0.00470461],
       [-0.05870989, -0.07404877,  0.17951696, ..., -0.00530585,
        -0.070803  , -0.01620434],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)>, 'tgt': <tf.Tensor: id=873, shape=(20, 2248), dtype=float32, numpy=
array([[-0.02069692, -0.0636159 ,  0.12213112, ...,  0.04802324,
        -0.03753156, -0.09227972],
       [-0.02587465, -0.07516001,  0.05057174, ...,  0.02407737,
        -0.03481697, -0.0290745 ],
       [ 0.        ,  0.        ,  

In [75]:
for data in dataset.map(func).batch(32).take(1):
    print(data)

{'src': <tf.Tensor: id=908, shape=(32, 20, 2248), dtype=float32, numpy=
array([[[-2.22197957e-02, -3.70566472e-02,  1.30662993e-01, ...,
         -1.36850914e-02, -5.11631854e-02,  1.39387576e-02],
        [-6.22378215e-02, -1.34056255e-01,  1.65300593e-01, ...,
          1.08687067e-03, -4.64779474e-02,  4.70461091e-03],
        [-5.87098859e-02, -7.40487725e-02,  1.79516956e-01, ...,
         -5.30584808e-03, -7.08030015e-02, -1.62043385e-02],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-3.62226297e-03,  1.06784711e-02,  1.58216611e-01, ...,
          8.72655865e-03,  1.18833687e-02, -4.91120107e-02],
        [-1.19733706e-01, -1.19

In [76]:
data["src"].numpy().shape

(32, 20, 2248)

In [77]:
data["tgt"].numpy().shape

(32, 20, 2248)

In [78]:
data["label"].numpy().shape

(32,)